<a href="https://colab.research.google.com/github/RomaricKanyamibwa/Normalisation-de-concepts-medicaux/blob/master/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip3 install --upgrade gensim

    100% |████████████████████████████████| 23.6MB 1.4MB/s 
    100% |████████████████████████████████| 1.4MB 13.9MB/s 
    100% |████████████████████████████████| 133kB 27.9MB/s 
    100% |████████████████████████████████| 61kB 21.8MB/s 
    100% |████████████████████████████████| 4.9MB 5.6MB/s 
    100% |████████████████████████████████| 552kB 23.4MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [0]:

import pandas as pd
#import collections
from pprint import pprint  # pretty-printer
import gensim
from gensim.models import word2vec
import logging

In [0]:
df_disorders=pd.read_csv("Disorders.csv",names=["Code","Disorder","GenDisorder"],sep="\t") 
Codes=df_disorders['Code']#Y vector (outputs)
Disorders=df_disorders['Disorder']#X vector(test)
GenDisorders=df_disorders['GenDisorder']#X vector (train)